<a href="https://colab.research.google.com/github/Voolshara/Nti_AI_academy_2020/blob/main/BERT_RTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets


     |████████████████████████████████| 163kB 16.2MB/s 
     |████████████████████████████████| 245kB 30.8MB/s 
     |████████████████████████████████| 17.7MB 204kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!pip install -U transformers

     |████████████████████████████████| 1.5MB 11.8MB/s 
     |████████████████████████████████| 890kB 58.5MB/s 
     |████████████████████████████████| 2.9MB 58.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ab8a49af20dd82208dcdfaef8733c02e642a2ffa83dec6bb43fa1727864e4496
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [ ]:
task = "rte"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task

In [ ]:
import torch

In [ ]:
!pip install jsonlines

In [ ]:
import json
import jsonlines

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl

--2021-01-08 10:00:36--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2930172 (2.8M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]   2.79M  --.-KB/s    in 0.05s   

2021-01-08 10:00:36 (61.0 MB/s) - ‘train.jsonl’ saved [2930172/2930172]



In [ ]:
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl

--2021-01-08 10:00:37--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 566932 (554K) [text/plain]
Saving to: ‘val.jsonl’

val.jsonl           100%[===================>] 553.64K  --.-KB/s    in 0.02s   

2021-01-08 10:00:37 (28.9 MB/s) - ‘val.jsonl’ saved [566932/566932]



In [ ]:
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/test.jsonl

--2021-01-08 10:00:37--  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/test.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1739290 (1.7M) [text/plain]
Saving to: ‘test.jsonl’

test.jsonl          100%[===================>]   1.66M  --.-KB/s    in 0.04s   

2021-01-08 10:00:37 (41.9 MB/s) - ‘test.jsonl’ saved [1739290/1739290]



In [ ]:
def get_RTE(data_json_file):
    X = dict()
    S1 = []
    S2 = []
    L = []
    I = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    #X.append('Text: ' +text+' Query: '+query+' Answer: '+a['text'])
                    #R = dict()
                    #R["sentence1"] = text
                    #R["sentence2"] = "Query: " +query+ " Answer: " + a['text']
                    #R["label"] = a['label']
                    S1.append(text)
                    S2.append(query + a['text'])
                    L.append(a['label'])
                    I.append(a['idx'])
    X["sentence1"] = S1
    X["sentence2"] = S2
    X["label"] = L
    X["idx"] = I               
    return X

In [ ]:
Train = get_RTE('train.jsonl')
Eval = get_RTE('val.jsonl')

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], padding=True, truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], padding=True, truncation=True)

In [ ]:
train_encodings, train_labels = preprocess_function(Train), Train["label"]

In [ ]:
val_encodings, val_labels = preprocess_function(Eval), Eval["label"]

In [ ]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=12,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=12,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=350,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

In [ ]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  

In [ ]:
trainer.train()

Step,Training Loss
500,0.692205
1000,0.691041
1500,0.688273
2000,0.689590
2500,0.687984
3000,0.690200
3500,0.690139
4000,0.687826
4500,0.689608
5000,0.689745


TrainOutput(global_step=8964, training_loss=0.6753471915820836)

In [ ]:
trainer.evaluate()

{'epoch': 12.0, 'eval_loss': 0.6568959355354309}

In [ ]:
  def get_Pred_RTE(data_json_file):
    X = []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            #print(item['passage'].keys())
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                   #X.append('Text: ' +text+' Query: '+query+' Answer: '+a['text'])
                   R = dict()
                   R["sentence1"] = text
                   R["sentence2"] = query + a['text']
                   X.append(R)         
    return X

In [ ]:
Predict = get_Pred_RTE('test.jsonl') 

In [ ]:
torch.cuda.set_device(0)

In [ ]:
model = model.to("cuda:0")

In [ ]:
model = torch.nn.DataParallel(model.cuda(), device_ids=[0])

In [ ]:
RTE = []
i = 0
for el in Predict:
  token = tokenizer(el[sentence1_key], el[sentence2_key], padding=True, truncation=True, return_tensors="pt")
  out = model(**token).logits
  ind = torch.softmax(out, dim=1).tolist()[0]
  RTE.append(ind)
  #print(ind)
  i += 1  
  if i % 500 == 0:
    print(i, "Predictions")  

500 Predictions
1000 Predictions
1500 Predictions
2000 Predictions
2500 Predictions
3000 Predictions
3500 Predictions
4000 Predictions
4500 Predictions
5000 Predictions
5500 Predictions
6000 Predictions
6500 Predictions
7000 Predictions
7500 Predictions
0 0


In [ ]:
file = open("out.txt", "w")
for el in RTE:
  file.write(str(el) + '\n')
file.close()  

In [ ]:
kol0 = 0
kol1 = 0
A = []
for ind in RTE:
  if ind[0] >= 0.574103:
    kol0 += 1
    A.append(0)
  else:
    if ind[1] <= 0.425:
      kol0 += 1
      A.append(0)
    else:
      A.append(1)
      kol1 += 1
print(kol0, kol1) # 1709 5905

1412 6202


In [ ]:
def raz(r):
  if r == 1:
    return 0
  return 1  

In [ ]:
All_Dict = []
i = 0
with jsonlines.open('test.jsonl') as reader:
    for jsn in reader:
        Main_Dict = dict()
        passage = jsn["passage"]
        Main_Dict['idx'] = jsn["idx"]
        Passage = dict()
        Passage["text"] = str(passage["text"])
        out2_arr = []
        for ques in passage["questions"]:
            qu = dict()
            qu["question"] = str(ques["question"])
            answers = ques["answers"]
            out3_arr = []
            for el in answers:
                out3 = dict()
                out3["idx"] = el["idx"]
                out3["text"] = str(el["text"])
                #out3["label"] = raz(A[i])
                out3["label"] = A[i]
                out3_arr.append(out3)
                i += 1
            qu["answers"] = out3_arr
            qu["idx"] = ques["idx"]
            out2_arr.append(qu)
        Passage["questions"] = out2_arr
        Main_Dict["passage"] = Passage
        jn = str(Main_Dict)
        #print(jn)
        All_Dict.append(Main_Dict)
with jsonlines.open('output.jsonl', 'w') as writer:
    writer.write_all(All_Dict)